In [22]:
##############
# Minimizers #
##############

# Note: All minimizer functions returns an array L such that
# L[1] = x^*, or the optimal point
# L[2] = value of the objective function at the optimal point
# L[3] = number of iterations used or DEFAULT_MAX_ITER

const DEFAULT_MAX_ITER = 100
const DEFAULT_TOLERANCE = 0.01

# Minimization algorithm using Newton's method
# f  the objective function
# g  the gradient of f
# h  the Hessian of f
# x  an initial testing point
# tolerance a small number
function minimizenewt(f,g, H, x::Vector, tolerance=DEFAULT_TOLERANCE, max_iter = DEFAULT_MAX_ITER)
    alpha =0.3
    beta = 0.7
    iter = 0
    
    # Default mode is RELATIVE tolerance, comment out the line below for ABSOLUTE tolerance
    tolerance = tolerance*norm(g(x))

    while norm(g(x)) > tolerance && iter < max_iter
        dx = -H(x)\g(x)
        t = 1
        while f(x+t*dx) > f(x) + alpha*t*sum(g(x).*dx)
            t = beta*t
        end
        x = x + t*dx
        iter = iter + 1
    end
    if iter < max_iter
        success =1
    else
        success =0
    end
    if iter > max_iter
        error("Exceeding max iterations")
    end
    return x
end




# Minimization algorithm using Newton's method, with fix for nonpositive definite matrix
# f  the objective function
# g  the gradient of f
# h  the Hessian of f
# x  an initial testing point
# tolerance a small number
function minimizenewtnpd(f,g, H, x::Vector, tolerance=DEFAULT_TOLERANCE, max_iter = DEFAULT_MAX_ITER)
    alpha =0.3
    beta = 0.7
    iter = 0
    function fixnonposdefmat(A, epsilon=0.1)
        M = eigfact(A)
        V = M[:vectors]
        L = M[:values]
        X = [max(abs(vi), epsilon)::Float64 for vi in L]
        L = diagm(X)
        return (V'*L*V)
    end
    
    # Default mode is RELATIVE tolerance, comment out the line below for ABSOLUTE tolerance
    tolerance = tolerance*sqrt(sum(g(x).^2))

    while sqrt(sum(g(x).^2)) > tolerance && iter < max_iter
        if isposdef(H(x))
            HH = H(x)
        else
            HH = fixnonposdefmat(H(x))
        end
        dx = -HH\g(x)
        t = 1
        while f(x+t*dx) > f(x) + alpha*t*sum(g(x).*dx)
            t = beta*t
        end
        x = x + t*dx
        iter = iter + 1
    end
    if iter > max_iter
        error("Exceeding max iterations")
    end
    println(f(x))
    println("iterations: $iter")
    return x
end




# Minimization algorithm using Newton's method, with fix for nonpositive definite matrix, version 2
# f  the objective function
# g  the gradient of f
# h  the Hessian of f
# x  an initial testing point
# tolerance a small number
function minimizenewtnpd2(f,g, H, x::Vector, tolerance=DEFAULT_TOLERANCE, max_iter = DEFAULT_MAX_ITER)
    alpha =0.3
    beta = 0.7
    iter = 0
    function fixnonposdefmat(A, epsilon=0.1)
        M = eigfact(A)
        V = M[:vectors]
        L = M[:values]
        X = [max(vi, epsilon)::Float64 for vi in L]
        L = diagm(X)
        return (V'*L*V)
    end
    
    # Default mode is RELATIVE tolerance, comment out the line below for ABSOLUTE tolerance
    tolerance = tolerance*norm(g(x))

    while norm(g(x)) > tolerance && iter < max_iter
        if isposdef(H(x))
            HH = H(x)
        else
            HH = fixnonposdefmat(H(x))
        end
        dx = -HH\g(x)
        t = 1
        while f(x+t*dx) > f(x) + alpha*t*sum(g(x).*dx)
            t = beta*t
        end
        x = x + t*dx
        iter = iter + 1
    end
    if iter > max_iter
        error("Exceeding max iterations")
    end
    return x
end







# Minimization algorithm using BFGS
# f  the objective function
# g  the gradient of f
# H  the initial guess of the INVERSE HESSIAN, you should set it to the identify matrix if you don't know
# x  an initial testing point
# tolerance a small number
function minimizeBFGS(f,g, H, x::Vector, tolerance=DEFAULT_TOLERANCE, max_iter = DEFAULT_MAX_ITER)
    alpha =0.3
    beta = 0.7
    iter = 0
    
    I = eye(length(x))
    
    # Default mode is RELATIVE tolerance, comment out the line below for ABSOLUTE tolerance
    # tolerance = tolerance*norm(g(x))

    while norm(g(x)) > tolerance && iter < max_iter
        dx = -H*g(x)
        t = 1
        while f(x+t*dx) > f(x) + alpha*t*sum(g(x).*dx)
            t = beta*t
        end
        x1 = x + t*dx
        s = t*dx
        y = g(x1) - g(x)
        rho = 1/dot(s,y)
        H = (I- rho* s *(y'))*H*(I-rho*y*(s')) + rho*s*(s')
        x = x1
        iter = iter + 1
    end
    if iter > max_iter
        error("Exceeding max iterations")
    end
    return x
end


# Minimization algorithm of equality constraint problem using Augmented Lagrangian
# f cost function
# g gradient of f, as a size n column vector, where n is the dimension of the domain
# c equality constraint, as a column function from R^n to R^m. c = [c_1;...;c_m] where c_i are real functions
# J the Jacobian matrix of c, m columns by n rows matrix, the i-th column should be the column gradient of c_i
#       Hence, multiplying by J on the left is a function from R^m to R^n
function minimizeAL(f,g, c, J, x, y, stopping_condition = DEFAULT_TOLERANCE)
    rho = 1
    eta = 0.6
    # n by n identity matrix
    I = eye(length(x))
    
    # iteration
    k = 0

    function converged()
        norm(c(x)) < stopping_condition && norm(g(x)- J(x)*y) < stopping_condition
    end
    while converged() == false
        # Augmented Lagrangian (AL) function
        L(x) = f(x) - dot(c(x),y) + (rho/2)* norm(c(x))^2
        # Derivative of the AL function with respect to x
        Lx(x) = g(x) - J(x)*(y- rho *c(x))

        x = minimizeBFGS(L, Lx, I, x, eta^k)
        if norm(c(x)) < eta^k
            y = y - rho*c(x)
        else
            rho = 10*rho
        end
        k+=1
    end
    @printf "Augmented Lagrangian with quasi-Newton method converged in \n %d iterations\n" k
    return (x,y)
end


minimizeAL (generic function with 2 methods)

In [23]:
# Hock-Schittkowski problem 6
f(x) = (1-x[1])^2
g(x) = [-2(1-x[1]);0]
c(x) = 10(x[2]-x[1]^2)
J(x) = [-20*x[1];10]
x = [-1.2;1]
y = 0
(x,y) = minimizeAL(f,g,c,J,x,y)
@printf "x* = "
println(x)
@printf "f(x*) = %f \n" f(x)

Augmented Lagrangian with quasi-Newton method converged in 
 8 iterations
x* = [0.9994304481550594,0.9992496873366561]
f(x*) = 0.000000 


In [21]:
# Hock-Schittkowski problem 7
f(x) = log(1+x[1]^2) - x[2]
g(x) = [2x[1]/(1+x[1]^2);-1]
c(x) = (1+x[1]^2)^2 + x[2]^2 - 4
J(x) = [2*(1+x[1]^2)*(2x[1]);2x[2]]
x = [2;2]
y = 0
(x,y) = minimizeAL(f,g,c,J,x,y)

@printf "x* = "
println(x)
@printf "f(x*) = %f \n" f(x)


Augmented Lagrangian with quasi-Newton method converged in 
 10 iterations
x* = [0.001186499998039978,1.7328983297573237]
f(x*) = -1.732897 


In [11]:
# Hock-Schittkowski problem 8
f(x) = -1
g(x) = [0,0]
c(x) = [x[1]^2+x[2]^2 - 25, x[1]*x[2] - 9]
J(x) = [2x[1] x[2] ; 2x[2] x[1]]
x = [2,1]
y = [0,0]
(x,y) = minimizeAL(f,g,c,J,x,y)

@printf "x* = "
println(x)
@printf "f(x*) = %f \n" f(x)

Quasinewton method converged in 9 iterations
[4.60253602813184,1.9545088015061312]
-1


In [12]:
# Hock-Schittkowski problem 9
f(x) = sin(pi*x[1]/12)*cos(pi*x[2]/16)
g(x) = [(pi/12)*cos(pi*x[1]/12)*cos(pi*x[2]/16),-(pi/16)*sin(pi*x[1]/12)*sin(pi*x[2]/16)]
c(x) = 4x[1] - 3x[2]
J(x) = [4;-3]
x = [2,1]
y = 0
(x,y) = minimizeAL(f,g,c,J,x,y)

@printf "x* = "
println(x)
@printf "f(x*) = %f \n" f(x)

Quasinewton method converged in 9 iterations
[20.915646594121846,27.888684794562934]
-0.4996308791289964
